<a href="https://colab.research.google.com/github/karthik111/video_anomaly_detection/blob/master/notebook/i3d_nl10_resnet50_v1_kinetics400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dmlc/gluon-cv/blob/onnx/scripts/onnx/notebooks/action-recognition/i3d_nl10_resnet50_v1_kinetics400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install --upgrade onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


In [2]:

import numpy as np
import onnxruntime as rt
import urllib.request
import os.path



(Optional) We use mxnet and gluoncv to read in video frames.

Feel free to read in video frames your own way


In [3]:
!pip3 install --upgrade mxnet gluoncv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [4]:

def fetch_model():
    if not os.path.isfile("i3d_nl10_resnet50_v1_kinetics400.onnx"):
        urllib.request.urlretrieve("https://apache-mxnet.s3-us-west-2.amazonaws.com/onnx/models/gluoncv-i3d_nl10_resnet50_v1_kinetics400-fce76386.onnx", filename="i3d_nl10_resnet50_v1_kinetics400.onnx")
    return "i3d_nl10_resnet50_v1_kinetics400.onnx"

def prepare_video(video_path, input_shape):
    from gluoncv.data.transforms import video
    from gluoncv.utils.filesystem import try_import_decord

    decord = try_import_decord()
    vr = decord.VideoReader(video_path)
    frame_id_list = list(range(0, 64, 2))
    num_frames = len(frame_id_list)
    video_data = vr.get_batch(frame_id_list).asnumpy()
    clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
    transform_fn = video.VideoGroupValTransform(size=input_shape[3],
                                                    mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])
    clip_input = transform_fn(clip_input)
    clip_input = np.stack(clip_input, axis=0)
    clip_input = clip_input.reshape((-1,) + (num_frames, 3, input_shape[3], input_shape[4]))
    clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

    return clip_input

def prepare_label():
    from gluoncv.data import Kinetics400Attr
    return Kinetics400Attr().classes


**Make sure to replace the video you want to use**

In [6]:

model = fetch_model()
video_path = '/content/archery.mp4'
clip_input = prepare_video(video_path, (1, 3, 32, 224, 224))
label = prepare_label()


RuntimeError: ignored

In [ ]:

# Create a onnx inference session and get the input name
onnx_session = rt.InferenceSession(model, None)
input_name = onnx_session.get_inputs()[0].name


In [ ]:

pred = onnx_session.run([], {input_name: clip_input.astype('float32')})[0]



(Optional) We use mxnet to process the result.

Feel free to process the result your own way


In [ ]:
!pip3 install --upgrade mxnet

In [ ]:

import mxnet as mx

pred = mx.nd.array(pred)
topK = 5
ind = mx.nd.topk(pred, k=topK)[0].astype('int')
print('The input is classified to be')
for i in range(topK):
    print('	[%s], with probability %.3f.'%
          (label[ind[i].asscalar()], mx.nd.softmax(pred)[0][ind[i]].asscalar()))


In [5]:
import torch
print(torch.__version__)

2.1.0+cu121
